In [ ]:
#default_exp problem

In [ ]:
#exporti
import copy
import torch
import numpy as np
from typing import Union

from dl4to.problem import PlottingForProblem, InputCheckerForProblem
from dl4to.topo_solvers import TrivialSolver

In [ ]:
#hide
from nbdev.showdoc import show_doc

# Problem

The Problem class contains all information of the underlying topology optimization (TO) problem one intends to solve. 
We focus on isotropic materials that are linearly elastic. This comprises most common materials, e.g., including steel and aluminum. Since we perform optimization on structured grids, all information is either in scalar or in tensor form. This makes data compatible with DL applications since it allows for a shape-consistent tensor representation. Let $(n_x, n_y, n_z)$ be the number of voxels in each spacial direction, i.e. the shape of the TO problem. 
    
We can create unique problem objects characterized by the following inputs:

- Scalar material properties that define the physical proporties of the underlying material. These include:
    - Young's modulus $E>0$
    - Poisson's ratio $\nu\in [0, 0.5]$
    - The yield stress $\sigma_{ys}>0$
    
- A three-dimensional vector $h$ that defines the voxel sizes in meters in each direction.

- A binary ($3\times n_x \times n_y \times n_z$)-tensor called $\Omega_\text{dirichlet}$ which we use to encode the presence of directional homogeneous Dirichlet boundary conditions for every voxel. These boundary conditions determine where the structure is "locked" in place, i.e. where the displacements are fixed at 0. $1$s indicate the presence, and $0$s the absence of homogeneous Dirichlet boundary conditions. Currently, we do not support non-homogeneous Dirichlet boundary conditions (i.e. voxels that have displacements fixed at some value $\neq 0$) since we believe that they are not required for most TO tasks.

- A ($1\times n_x \times n_y \times n_z$)-tensor called $\Omega_\text{design}$ containing values $\in \left\lbrace 0,1,-1\right\rbrace$ that we use to encode design space information. We use $0$s and $1$s to constrain voxel densities to be $0$ or $1$, respectively. Entries of $-1$ indicate a lack of density constraints, which signifies that the density in that voxel can be freely optimized.

- A ($3\times n_x \times n_y \times n_z$)-tensor called $F$, which encodes external forces given in $\text{N}/\text{m}^3$. The three channels correspond to the force magnitudes in each spacial dimension. For voxels that have external loads assigned to them we automatically enforce the corresponding density value to be $1$.

In [ ]:
#export
class Problem:
    """
    A class containing all parameters for defining a topology optimization problem.
    """
    def __init__(
        self, 
        E:float, # The Young's modulus of the material. Given in Pa.
        ν:float, # The Poisson's ratio of the material. Dimensionless.
        σ_ys:float, #The yield stress σ_ys denotes the critical von Mises stress at which the material starts yielding. Given in Pa.
        h:Union[float,list], # The length of the edges of the cuboid voxels. Equal to the discretisation step size in each coordinate direction.
        Ω_dirichlet:torch.Tensor, #A tensor denoting the presence of homogeneous Dirichlet boundary conditions in each voxel in each coordinate direction. 
        Ω_design:torch.Tensor, # A tensor denoting the kind of design space assigned to each voxel. Values of "0" and "1" indicate a material density fixed at 0 or 1, respectively. "-1" indicates the absence of constraints, i.e., the voxel density can be freely optimized.
        F:torch.Tensor, # A tensor denoting the forces applied to each voxel in each coordinate direction. Given in N/m^3.
        pde_solver:"dl4to.pde.PDESolver"=None, # A dl4to PDE Solver object that is attached to this problem.
        name:str=None, # The name of the problem
        device:str='cpu', # The device that this problem is to be stored on. Possible options are "cpu" and "cuda".
        dtype:torch.dtype=torch.float32, # The datatype of the problem.
        restrict_density_for_voxels_with_applied_forces:bool=True # Determines whether Ω_design should be set to "1" for voxels that have forces applied to them. Should be turned of in case of volumetric forces like gravity that are applied to all voxels.
    ):
        self._dtype = dtype
        self._device = device
        self._E, self._ν, self._σ_ys = E, ν, σ_ys
        if type(h) == list or type(h) == np.ndarray:
            h = torch.tensor(h)
        if type(h) == int or type(h) == float:
            h = torch.tensor([h,h,h])
        self._h = h
        self._Ω_dirichlet = Ω_dirichlet.type(torch.bool).to(device)
        self._Ω_design = Ω_design.type(dtype).to(device)
        self._F = F.type(dtype).to(device)
        self._shape = self.Ω_design.shape[-3:]
        self._size = (torch.tensor(self.shape) * self.h).tolist()
        if restrict_density_for_voxels_with_applied_forces:
            F_mask = (self.F != 0).sum(dim=0, keepdim=True).bool()
            self._Ω_design[F_mask] = 1.
        self._name = name
        self.trivial_solution = TrivialSolver()(self)
        self.pde_solver = pde_solver
        InputCheckerForProblem.check_init(problem=self)


    @property
    def pde_solver(self):
        return self._pde_solver


    @pde_solver.setter
    def pde_solver(self, pde_solver):
        if pde_solver is None:
            self._pde_solver = pde_solver
        else:
            self._pde_solver = pde_solver.clone()
            if self._pde_solver.assemble_tensors_when_passed_to_problem:
                self._pde_solver.assemble_tensors(self)
            self.trivial_solution.u = None
            self.trivial_solution.u_binary = None


    @property
    def name(self):
        return self._name


    @property
    def dtype(self):
        return self._dtype


    @dtype.setter
    def dtype(self, dtype):
        self._dtype = dtype
        self._Ω_design = self._Ω_design.type(dtype)
        self._F = self._F.type(dtype)
        if self.pde_solver is not None:
            self.pde_solver.dtype = dtype


    @property
    def h(self):
        return self._h


    @property
    def E(self):
        return self._E


    @property
    def ν(self):
        return self._ν


    @property
    def σ_ys(self):
        return self._σ_ys


    @property
    def Ω_dirichlet(self):
        return self._Ω_dirichlet


    @property
    def Ω_design(self):
        return self._Ω_design


    @property
    def F(self):
        return self._F


    @property
    def shape(self):
        return self._shape


    @property
    def size(self):
        return self._size


    @property
    def device(self):
        return self._device


    @device.setter
    def device(self, device):
        self._Ω_dirichlet = self._Ω_dirichlet.to(device)
        self._Ω_design = self._Ω_design.to(device)
        self._F = self._F.to(device)
        self._device = device
        if self.pde_solver is not None:
            self.pde_solver.device = device


    def clone(self):
        """
        Returns a deepcopy of the Problem object.
        """
        return copy.deepcopy(self)


    def plot(self, 
             display:bool=True, # Whether the figure is displayed.
             file_path:str=None, # Path where the figure is saved.
             camera_position:Union[list,tuple]=(0,.1,.12), # x, y, and z coordinates of the camera position.
             use_pyvista:bool=False, # Whether to use pyvista for plotting. If `False`, then plotly is used. Pyvista generates better looking visualizations, but does not support basic features like colorbars, title display and saving.
             window_size:Union[tuple,list]=(800,800), # The size of the window that displays the plot. Only has an effect if `use_pyvista=True`.
             smooth_iters:int=0, # The number of smoothing iterations for better looking visualizations. Only has an effect if `use_pyvista=True`.
             show_colorbar:bool=True, # Determines whether a reference colorbar is displayed for the plotted voxel color values.
             show_axislabels:bool=False, # Whether the 3d axes are labelled with their dimensions.
             show_ticklabels:bool=False, # Whether the 3d axes ticks are displayed and labeled.
             export_png:bool=False): # Whether the figure is exported and saved as a png file, in addition to the standard html format.
        """
        Renders 3D figures that display the location of Dirichlet boundaries, design space and forces.
        """
        PlottingForProblem()(
            self,
            display=display, 
            file_path=file_path, 
            camera_position=camera_position, 
            use_pyvista=use_pyvista,
            window_size=window_size,
            smooth_iters=smooth_iters,
            show_colorbar=show_colorbar,
            show_axislabels=show_axislabels, 
            show_ticklabels=show_ticklabels, 
            export_png=export_png,
        )

## Methods

In [ ]:
show_doc(Problem.clone)

<h4 id="Problem.clone" class="doc_header"><code>Problem.clone</code><a href="__main__.py#L137" class="source_link" style="float:right">[source]</a></h4>

> <code>Problem.clone</code>()

Returns a deepcopy of the Problem object.

In [ ]:
show_doc(Problem.plot)

<h4 id="Problem.plot" class="doc_header"><code>Problem.plot</code><a href="__main__.py#L144" class="source_link" style="float:right">[source]</a></h4>

> <code>Problem.plot</code>(**`display`**:`bool`=*`True`*, **`file_path`**:`str`=*`None`*, **`camera_position`**:`Union`\[`list`, `tuple`\]=*`(0, 0.1, 0.12)`*, **`use_pyvista`**:`bool`=*`False`*, **`window_size`**:`Union`\[`tuple`, `list`\]=*`(300, 300)`*, **`smooth_iters`**:`int`=*`0`*, **`show_colorbar`**:`bool`=*`True`*, **`show_axislabels`**:`bool`=*`False`*, **`show_ticklabels`**:`bool`=*`False`*, **`export_png`**:`bool`=*`False`*)

Renders 3D figures that display the location of Dirichlet boundaries, design space and forces.

||Type|Default|Details|
|---|---|---|---|
|**`display`**|`bool`|`True`|Whether the figure is displayed.|
|**`file_path`**|`str`|`None`|Path where the figure is saved.|
|**`camera_position`**|`typing.Union[list, tuple]`|`(0, 0.1, 0.12)`|x, y, and z coordinates of the camera position.|
|**`use_pyvista`**|`bool`|`False`|Whether to use pyvista for plotting. If `False`, then plotly is used. Pyvista generates better looking visualizations, but does not support basic features like colorbars, title display and saving.|
|**`window_size`**|`typing.Union[tuple, list]`|`(300, 300)`|The size of the window that displays the plot. Only has an effect if `use_pyvista=True`.|
|**`smooth_iters`**|`int`|`0`|The number of smoothing iterations for better looking visualizations. Only has an effect if `use_pyvista=True`.|
|**`show_colorbar`**|`bool`|`True`|Determines whether a reference colorbar is displayed for the plotted voxel color values.|
|**`show_axislabels`**|`bool`|`False`|Whether the 3d axes are labelled with their dimensions.|
|**`show_ticklabels`**|`bool`|`False`|Whether the 3d axes ticks are displayed and labeled.|
|**`export_png`**|`bool`|`False`|Whether the figure is exported and saved as a png file, in addition to the standard html format.|


## Examples

In [ ]:
#ignore
import torch

# scalar material properties
E = 7e10 # Young's modulus (in Pascal)
ν = .3 # Poisson's ratio
σ_ys = 4.5e8 # Yield stress (in Pascal)

shape = [40, 4, 8] # number of voxels in x, y and z direction
h = [0.0250, 0.0250, 0.0250] # spacial dimensions of each voxel in x, y and direction (in meters)

# define external forces
F = torch.zeros(3, *shape)
F[-1, :, :, -1] = -4e7 # external forces (in Newton)

# define locations of homogeneous Dirichlet conditions
Ω_dirichlet = torch.zeros(3, *shape)
Ω_dirichlet[:, 0, :, :] = 1

# define design space
Ω_design = -torch.ones(1, *shape) # -1s indicate that these voxels can be freely optimized
Ω_design[:,:2,:,:] = 1. # we set densities to 1 where there are Dirichlet boundary conditions
Ω_design[:,:,:,-2:] = 1. # we set densities to 1 where there are external forces

problem = Problem(E, ν, σ_ys, h, Ω_dirichlet, Ω_design, F)

In [ ]:
#ignore
camera_position = (0, 0.35, 0.2)
problem.plot(camera_position=camera_position,
             show_axislabels=True,
             show_ticklabels=True,
             display=False)

![dirichlet](https://dl4to.github.io/dl4to/images/1_dirichlet.png)
![design](https://dl4to.github.io/dl4to/images/1_design.png)
![force_loc](https://dl4to.github.io/dl4to/images/1_force_locs.png)
![force_dir](https://dl4to.github.io/dl4to/images/1_force_dirs.png)

In [ ]:
#hide 
def get_problem(device='cpu', dtype=torch.float32):
    name = 'testname'
    h = torch.ones(3)
    E, ν = 1, 1

    Ω_dirichlet = torch.zeros(3, 5, 5, 5)
    Ω_design = torch.zeros(1, 5, 5, 5)
    F = torch.rand(3, 5, 5, 5)

    problem = Problem(
        E=E, ν=ν, σ_ys=2.5e8, h=h,
        Ω_dirichlet=Ω_dirichlet,
        Ω_design=Ω_design,
        F=F,
        name='testname',
        device=device,
        dtype=dtype,
    )

    return problem

In [ ]:
%%time
#hide



def test_device_and_dtype():
    device = 'cpu'
    for dtype in [torch.float, torch.float64]:
        problem = get_problem(device=device, dtype=dtype)

        assert problem.Ω_dirichlet.device.type == device
        assert problem.Ω_design.device.type == device
        assert problem.F.device.type == device
        assert problem.F.dtype == dtype

test_device_and_dtype()

CPU times: user 2.2 ms, sys: 0 ns, total: 2.2 ms
Wall time: 2.01 ms


In [ ]:
%%time
#hide

def test_that_the_clone_has_the_same_attributes():
    problem = get_problem()
    problem_clone = problem.clone()

    assert torch.all(problem.Ω_dirichlet == problem_clone.Ω_dirichlet)
    assert torch.all(problem.Ω_design == problem_clone.Ω_design)
    assert torch.all(problem.F == problem_clone.F)
    assert problem.E == problem_clone.E
    assert problem.ν == problem_clone.ν
    assert problem.name == problem_clone.name


test_that_the_clone_has_the_same_attributes()

CPU times: user 1.72 ms, sys: 0 ns, total: 1.72 ms
Wall time: 1.58 ms


In [ ]:
%%time
#hide

def test_invalid_geometry_gets_caught():
    caught = True

    try:
        problem = Problem(
            name='testname',
            p0=np.array([0, 0, 0]),
            p1=np.array([1, -1, 1]),
            E=E, ν=ν,
            Ω_dirichlet=Ω_dirichlet,
            Ω_design=Ω_design,
            F=F
        )
        caught = False
    except:
        pass

    assert caught


test_invalid_geometry_gets_caught()

CPU times: user 40 µs, sys: 0 ns, total: 40 µs
Wall time: 43.2 µs


In [ ]:
%%time
#hide

def test_can_one_differentiate_in_Problem():
    F_ = torch.rand(3, 5, 5, 5, requires_grad=True)
    F = F_ * .5

    name = 'testname'
    h = torch.ones(3)
    E, ν = 1, 1

    Ω_dirichlet = torch.zeros(3, 5, 5, 5)
    Ω_design = torch.zeros(1, 5, 5, 5)

    problem = Problem(
        name='testname',
        h=h,
        E=E, ν=ν, σ_ys=2.5e8,
        Ω_dirichlet=Ω_dirichlet,
        Ω_design=Ω_design,
        F=F
    )

    assert problem.F.requires_grad

    grad = torch.autograd.grad(problem.F.mean(), F_)
    assert len(grad) == 1

    grad = grad[0]
    assert grad.shape == (3, 5, 5, 5)
    assert torch.all(grad == 0.004/3)


test_can_one_differentiate_in_Problem()

CPU times: user 2.49 ms, sys: 0 ns, total: 2.49 ms
Wall time: 2.08 ms


In [ ]:
%%time
#hide

def test_move_to_device(device='cpu'):
    problem = get_problem()
    problem.device = device
    assert str(problem.Ω_dirichlet.device) == device

test_move_to_device()

CPU times: user 924 µs, sys: 0 ns, total: 924 µs
Wall time: 841 µs
